In [41]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import binance as bn
import pandas as pd
import sys
from datetime import datetime, timedelta
import ta as ta
import numpy as np
import json
import sys
sys.path.append('C:\\Users\\Administrator\\Desktop\\utils')
import utils
import matplotlib.pyplot as plt

l_t: 2022-10-20 08:57:23.314000|st: 2022-10-20 08:55:00|l_p: 19119.09|macdd: -3.96|p: None |c: False


In [42]:
pd.set_option('display.max_rows', None)

l_t: 2022-10-20 08:57:25.316000|st: 2022-10-20 08:55:00|l_p: 19122.67|macdd: -3.70|p: None |c: False


In [43]:
class Long_macd_simp_trader():
    
    """Class to perform live testing using Binance testnet stream of data
    """
    def __init__(self, symbol=None, units='0.0006', interval=None, ema_slow=None, ema_fast=None, ema_signal=None, sesion_end=None, testnet=None, duration_minutes=None, emergency_price_decrease_pct=None):
        
        """Long_macd_trader class constructor
        :param interval: a string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :position: initial position in the market. 1 for long (have the asset) 0 for neutral (dont have the asset)
        :position: integer
        :back_testing: set to True if backtesting is necessary
        """
        self.units = units
        self.symbol = symbol
        self.interval = interval
        self.ema_slow = ema_slow
        self.ema_fast = ema_fast
        self.ema_signal = ema_signal 
        self.testnet = testnet #if the sesion takes place in the testnet
        self.duration_minutes = duration_minutes # duration of the trading sesion in minutes
        self.run_end_time_utc = None
        self.run_end_delta = None
        self.data = pd.DataFrame()
        self.trades = 0 #counter of the number of trades
        self.trade_values = [] #all trading positions consecutive, long/short... or short/long...
        self.trade_values_time = [] #time at which each value in trade_values was appended
        self.position = None #initially no position is decided, it is pending to analysis recent data of macd to decide
                             #what is the best way to enter the market        self.tct = None #turned complete time
        self.client = None 
        self.trade_start_time_utc = None # time in utc to be defined when the stream of OHLC starts ( this time
        #is used to extract the difference between actual time of trading sesion in utc time and the index of the
        # last kandle data that it is given also in utc time by binance api)
        self.twm = None # Initialize ws client
        self.profit = None #Final profit at the end of the trading sesion where the latest position is neutral
        self.trades_num = None
        self.initial_balance_USDT = None
        self.final_balance_USDT = None
        self.initial_balance_BTC = None
        self.final_balance_BTC = None
        self.cum_profits = None
        self.close_pair = [] #close price to implement the safety returns threshold
        self.emergency_price_decrease = False
        self.emergency_msg = None
        self.pct_price = None
        self.emergency_price_decrease_pct = emergency_price_decrease_pct
        
    
    def stream_candles(self, msg):
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time, 'Open'] = first
        self.data.loc[start_time, 'High'] = high
        self.data.loc[start_time, 'Low'] = low
        self.data.loc[start_time, 'Close'] = close
        self.data.loc[start_time, 'Volume'] = volume
        self.data.loc[start_time, 'Complete'] = complete
        # add balance_ini column with the balance in the latest bar before executing order (after complete==True)
        self.data.loc[start_time, 'balance_ini_USDT'] = np.nan
        self.data.loc[start_time, 'balance_ini_BTC'] = np.nan
        # initialize quote_units
        self.data.loc[start_time, 'quote_units'] = np.nan
        #update MACD parameters with each ws retrieval
        macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
        macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
        macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()         
        self.data.loc[start_time, 'macd_diff'] = macd_diff.iloc[-1]
        self.data.loc[start_time, 'macd_macd'] = macd_macd.iloc[-1]
        self.data.loc[start_time, 'macd_signal'] = macd_signal.iloc[-1]
        ##################################################
        #INTRODUCE SOME SORT OF EMERGENCY SELL DUE TO HIGH DROP IN PRICE IF WE ARE LONG.
        #IN THIS CASE IT IS NOT NECESSARY TO HAVE THE KANDLE COMPLETE TO EXECUTE THE MARKET SELL ORDER.
        
        if (len(self.close_pair) == 0):
            self.close_pair.append(close)
        if (len(self.close_pair) == 1):
            self.close_pair.insert(0,close)
        if(len(self.close_pair) == 2):
            self.close_pair.pop()
            self.close_pair.insert(0,close)
            self.pct_price = ((self.close_pair[1]/self.close_pair[0])-1)
            #print('testing safety percentatge, value is =', self.pct_price, self.close_pair[0], self.close_pair[1]) 
            if (self.pct_price < self.emergency_price_decrease_pct):
                self.emergency_price_decrease = True
                self.emergency_msg = 'PRICE DECREASE' 
                self.stop_ses()
            
        #FEATURE:#
        #LOOK FOR THE OUTLIERS OF NEGATIVE RETURNS IN BTC
        ##################################################
        
        ########################################################
        #IMPLEMENT SAME FEATURE BUT FOR PRICE INCREASE, GO LONG#
        ########################################################
        
       
        # print out
        print(f"l_t: {event_time}|st: {start_time}|l_p: {format(self.data.Close[-1],'.2f')}|macdd: {format(self.data.macd_diff[-1],'.2f')}|p: {self.position} |c: {complete}")      
        
        # print out
#         print(".", end = "", flush = True) # just print something to get a feedback (everything OK)
        dt = datetime.utcnow() - self.trade_start_time_utc
        if ((dt) > timedelta(minutes=self.duration_minutes)):
            self.stop_ses()
        
        if (complete == True):
            self.tct = start_time #time of the lastest updated/incorporated kandle            
            self.define_strategy()
            self.execute_trades()
                
    def start_trading(self):
        self.trade_start_time_utc = datetime.utcnow()
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        self.prepare_recent_data()
        self.initial_balance_USDT = round(float(self.client.get_asset_balance(asset='USDT')['free']),3)
        self.initial_balance_BTC = round(float(self.client.get_asset_balance(asset='BTC')['free']),3)
        
        try:                  
            
            self.twm.start_kline_socket(callback = self.stream_candles, symbol = self.symbol, interval = self.interval)
        
        except (BinanceAPIException, ConnectionResetError, requests.exceptions.ConnectionError, requests.exceptions.RequestException) as e:
            print(e)
            print('Something went wrong. Error occured at %s. The sesions will be automatically stopped after GOING NEUTRAL.' % (datetime.now().astimezone(timezone.utc)))
            self.stop_ses()
            
    def stop_ses(self, save_to_file=True):

        self.run_end_time_utc = datetime.utcnow()
        dt = self.run_end_time_utc - self.trade_start_time_utc
        self.run_end_delta = round(dt.seconds/60,0)
        print(f"trading sesion duration = {self.run_end_delta} minutes up to {self.duration_minutes}")
        if self.position == 0:
            if (self.emergency_price_decrease == True):
                print(f"EMERGENCY STOP. REASON: {self.emergency_msg}")
            else:
                print("STOP")
        if self.position == 1:
            if (self.emergency_price_decrease == True):
                order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, f"GOING NEUTRAL AND EMERGENCY STOP. REASON: {self.emergency_msg}") 
                self.position = 0
            else:
                order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP") 
                self.position = 0
        if self.position == None:
            if (len(self.trade_values) == 0):
                if (self.emergency_price_decrease == True):
                    print(f"EMERGENCY STOP (WITHOUT PLACING ORDERS). REASON: {self.emergency_msg}")
                else:
                    print("STOP BEFORE PLACING ORDERS")
            else:               
                if (self.trade_values[-1] < 0):
                    if (self.emergency_price_decrease == True):
                        order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                        self.report_trade(order, f"GOING NEUTRAL AND EMERGENCY STOP. REASON: {self.emergency_msg}") 
                        self.position = 0
                    else:
                        self.position = 0 # latest neutral position
                        li = self.data.index == self.data.index[-1]
                        self.data.loc[li, 'position'] = self.position 
                        order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                        self.report_trade(order, "GOING NEUTRAL AND STOP") 
                else:
                    if (self.emergency_price_decrease == True):
                        print(f"EMERGENCY STOP. REASON: {self.emergency_msg}")
                    else:
                        print("STOP")
        self.twm.stop()
        
        trades_num_filter = self.data.position.dropna() != None
        self.trades_num = len(trades_num_filter)
        start_ses = datetime.strftime(self.trade_start_time_utc, "%Y-%m-%d-%H:%M")
        li = self.data.index == self.data.index[-1]
        self.data.loc[li, 'position'] = self.position
        self.final_balance_USDT = round(float(self.client.get_asset_balance(asset='USDT')['free']),3)
        self.final_balance_BTC = round(float(self.client.get_asset_balance(asset='BTC')['free']),3)
        
        if (save_to_file == True):
            file_name = f"symbol_{self.symbol}__interval_{self.interval}__eslow_{self.ema_slow}_efast_{self.ema_fast}_esign_{self.ema_signal}__duration_{self.run_end_delta}min_upto_{self.duration_minutes}min__profit_{self.cum_profits}dollar__tradesnum_{self.trades_num}" 
            outfile = open(file_name, 'wb')
            self.data.to_csv(outfile, index = False, header = True, sep = ',', encoding = 'utf-8')
            outfile.close()    
    def define_strategy(self):
        ht_previous = self.data.macd_diff.iloc[-2]# this is the previous kandle to the just complete one
        ht_last = self.data.macd_diff.iloc[-1] #this is the last kandle that has just turned into a complete one
        if ((ht_previous > 0) and (ht_last < 0)):
            self.position = 0 #position adopted in the latest kandle that has turned into complete
        elif ((ht_previous < 0) and (ht_last > 0)):
            self.position = 1 #position adopted in the latest kandle that has turned into complete
        else:  
            self.position = None #in any other case do nothing and continue
        
        li = self.data.index == self.data.index[-1]
        self.data.loc[li, 'position'] = self.position    

    def execute_trades(self):
        cond_last = self.data.index == self.data.index[-1]
        #execute trades depending on the positions stablished in define_strategy()
        if self.position == 1:
            self.data.loc[cond_last, 'balance_ini_USDT'] = round(float(self.client.get_asset_balance(asset='USDT')['free']),3)
            self.data.loc[cond_last, 'balance_ini_BTC'] = round(float(self.client.get_asset_balance(asset='BTC')['free']),3)                        
            order = self.client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
            self.report_trade(order, "GOING LONG")

        if self.position == 0:
            self.data.loc[cond_last, 'balance_ini_USDT'] = round(float(self.client.get_asset_balance(asset='USDT')['free']),3)
            self.data.loc[cond_last, 'balance_ini_BTC'] = round(float(self.client.get_asset_balance(asset='BTC')['free']),3)                                    
            order = self.client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
            self.report_trade(order, "GOING NEUTRAL")

        if self.position == None:
            pass
        
    def report_trade(self, order, going):
        last_index = self.data.index[-1]
        cond_last = self.data.index == self.data.index[-1]
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        
        # calculate trading profits
        self.trades += 1
        
        last_t_str = datetime.strftime(self.data.index[-1], '%Y-%m-%d-%H:%M')
        
        if side == "BUY":
            self.trade_values.append(-quote_units)
            self.data.loc[cond_last, 'quote_units'] = -quote_units
        elif side == "SELL":
            self.trade_values.append(quote_units)
            self.data.loc[cond_last, 'quote_units'] = quote_units
        
        self.trade_values_time.append(last_t_str)
            
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)
           
        self.cum_profits = cum_profits
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, self.data.Close.iloc[-1]))
        print("{} | Real profit = {} | Accumulate profit = {} ".format(time, real_profit, cum_profits))
        if (self.emergency_price_decrease == True):
            print("price decreased in 1s in pct: {}, which is more than the imposed pct: {} ".format(self.pct_price, self.emergency_price_decrease_pct))        
        print(100 * "-" + "\n")
        
    def prepare_recent_data(self):
        '''
        REMARK: Introduced time must be in Tokyo time (UTC+9) but the calculations will be in UTC
        Prepare all the fields of dat a necessary for the study. The interval of dates to be studied is the one
        given when delclaring the class. To prepare another interval of dates, please create another class instance.
        :param start: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-01-29-20:00"
        :type start: str.
        :param end: a string with the following format ""%Y-%m-%d-%H:%M" .i.e. "2022-02-29-20:00"
        :type end: str.
        :param interval: string among the followings: ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        :type interval: str.
        '''
        
        #Prepare pre-data to the present moment with non NaN for the trading period (macd paramters ready)
        current_time_obj = datetime.now()
        current_time = int(current_time_obj.timestamp()*1000)

#         from_time_obj = datetime.strptime(start, "%Y-%m-%d-%H:%M")
#       ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        ema_diff = self.ema_slow + self.ema_signal + 2
        td = timedelta()
        if 'm' in self.interval:
            num_min = int(self.interval.replace('m',''))
            td = timedelta(minutes=num_min*(ema_diff))
        if 'h' in self.interval:
            num_h = int(self.interval.replace('h',''))
            td = timedelta(hours=num_h*(ema_diff))
        if 'd' in self.interval:
            num_day = int(self.interval.replace('d',''))
            td = timedelta(days=num_day*(ema_diff))
        if 'w' in self.interval:
            num_week = int(self.interval.replace('w',''))
            td = timedelta(weeks=num_week*(ema_diff))
        if 'M' in self.interval:
            num_week_m = int(self.interval.replace('M',''))
            td = timedelta(weeks=num_week_m * 4 * (ema_diff))
        from_time_obj = current_time_obj - td
        from_time = int((current_time_obj - td).timestamp()*1000)
        self.data, self.client = utils.get_history_v3(symbol=self.symbol, interval=self.interval, start=from_time, end=current_time, testnet=self.testnet)
        #obtaining MACD instance from python ta
        macd_diff = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_diff()
        macd_macd = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd()
        macd_signal = ta.trend.MACD(close=self.data.Close, window_slow=self.ema_slow, window_fast=self.ema_fast, window_sign=self.ema_signal, fillna=False).macd_signal()
        #assigning the values of macd to ticker dataframe
        self.data['macd_diff'] = macd_diff
        self.data['macd_macd'] = macd_macd
        self.data['macd_signal'] = macd_signal
        #last candle retrieved from old data till the present moment will be incomplete 99.9% of the time
        self.data["Complete"] = [True for row in range(len(self.data)-1)] + [False]
        #decide position to be given to the latest recent data kandle (not completed in 99% of cases and that is 
        #going to be updated by the stream of data, so actually, position given to the latest kandle at this point
        #it is mots likely not to be used, but updated.
        ht_previous = self.data.macd_diff.iloc[-2]# this is the kandle before the latest (complete in recent data)
        ht_last = self.data.macd_diff.iloc[-1] #this is the last (incomplete 99% cases)
        if ((ht_previous > 0) and (ht_last < 0)):
            self.position = 0 #position adopted in the latest kandle that is incomplete 99% cases
        elif ((ht_previous < 0) and (ht_last > 0)):
            self.position = 1 #position adopted in the latest kandle that is incomplete 99% cases
        else:  
            self.position = None #in any other case do nothing and continue
        
        li = self.data.index == self.data.index[-1]
        self.data.loc[li, 'position'] = self.position
    
    def open_orders(self):
        orders = self.client.get_open_orders(symbol=self.symbol)
        print(orders)
    
    def plot_results(self, start_plot=None, end_plot=None, width_bars=0.1):
        
        # from IPython.core.display import display, HTML
        # display(HTML("<style>.container { width:100% !important; }</style>"))
        colors=[]

        fig, (close_ax, macd_ax) = plt.subplots(nrows=2, ncols=1, figsize=(30,20), gridspec_kw={'height_ratios': [4,2]}, sharex=True)

        close_ax.grid(visible=True, which='major', axis='x', color='grey')
        macd_ax.grid(visible=True, which='major', axis='x', color='grey')
        close_ax.grid(visible=True, which='major', axis='y', color='grey')
        macd_ax.grid(visible=True, which='major', axis='y', color='grey')
        close_ax.grid(visible=True, which='minor', axis='x', color='grey')
        macd_ax.grid(visible=True, which='minor', axis='x', color='grey')

        close_ax.tick_params(labelrotation=45, labelsize = 'large')
        macd_ax.tick_params(labelrotation=45, labelsize = 'large')

        close_ax.margins(0)
        macd_ax.margins(0)
        
        close_ax.set_ylim(auto=True)
        
        data_ready = self.data.dropna(subset=['macd_diff', 'macd_macd', 'macd_signal']).copy()
        
        
        if (start_plot == None):
            start_plot = data_ready.index[0]
        if (end_plot == None):
            end_plot = data_ready.index[-1]  
            
        if ((start_plot != None) and (end_plot !=None)):
            cond_start = data_ready.index >= start_plot
            cond_end = data_ready.index <= end_plot
            data_ready = data_ready[cond_start&cond_end]
                
        for index, value in data_ready.macd_diff.iteritems():
            if value > 0:
                colors.append('g')
            else:
                colors.append('r')
                
        close_ax.plot(data_ready.index, data_ready.Close) #plot the data without shifting

        #shift one position the inv_sign only for plotting the signal in the day after is found, without shifting the
        #Close prices
        data_ready_shift = data_ready.copy()
        data_ready_shift['position'] = data_ready.position.shift(1)
        buy_pos = data_ready_shift.position == 1              
        buy_trade = data_ready_shift.loc[buy_pos]
        sell_pos = data_ready_shift.position == 0            
        sell_trade = data_ready_shift.loc[sell_pos]
        close_ax.scatter(sell_trade.index, sell_trade.Close.loc[sell_trade.index], marker='^', color='r', s=100)
        close_ax.scatter(buy_trade.index, buy_trade.Close.loc[buy_trade.index], marker='^', color='g', s=100)

        if ((self.data.Complete.iloc[-1] == False) and (self.data.position.iloc[-1] == 0)):
            close_ax.scatter(self.data.index[-1], self.data.Close.loc[self.data.index[-1]], marker='^', color='r', s=100)
        
        macd_ax.bar(x= data_ready.index, height= data_ready.macd_diff, width=width_bars, align='center', color=colors, edgecolor='black')
    
    def save_to_file(self):
        file_name = f"symbol_{self.symbol}__interval_{self.interval}__eslow_{self.ema_slow}_efast_{self.ema_fast}_esign_{self.ema_signal}__duration_{self.run_end_delta}min_upto_{self.duration_minutes}min__profit_{self.cum_profits}dollar__tradesnum_{self.trades_num}" 
        outfile = open(file_name, 'wb')
        self.data.to_csv(outfile, index = False, header = True, sep = ',', encoding = 'utf-8')
        outfile.close()              

l_t: 2022-10-20 08:57:27.461000|st: 2022-10-20 08:55:00|l_p: 19122.46|macdd: -3.72|p: None |c: False
l_t: 2022-10-20 08:57:29.545000|st: 2022-10-20 08:55:00|l_p: 19122.38|macdd: -3.72|p: None |c: False
l_t: 2022-10-20 08:57:31.549000|st: 2022-10-20 08:55:00|l_p: 19123.72|macdd: -3.63|p: None |c: False
l_t: 2022-10-20 08:57:33.755000|st: 2022-10-20 08:55:00|l_p: 19122.72|macdd: -3.70|p: None |c: False
l_t: 2022-10-20 08:57:35.883000|st: 2022-10-20 08:55:00|l_p: 19121.66|macdd: -3.77|p: None |c: False


In [39]:
#trader_test = Long_macd_simp_trader(symbol="BTCUSDT", units='0.05',interval='5m', ema_slow=25, ema_fast=12, ema_signal=25, testnet=True, duration_minutes=2880, emergency_price_decrease_pct=-0.05)

In [40]:
trader_test.start_trading()

52
l_t: 2022-10-19 07:08:03.752000|st: 2022-10-19 07:05:00|l_p: 19189.15|macdd: -9.52|p: None |c: False
l_t: 2022-10-19 07:08:05.808000|st: 2022-10-19 07:05:00|l_p: 19191.85|macdd: -9.33|p: None |c: False
l_t: 2022-10-19 07:08:07.822000|st: 2022-10-19 07:05:00|l_p: 19191.19|macdd: -9.38|p: None |c: False
l_t: 2022-10-19 07:08:09.844000|st: 2022-10-19 07:05:00|l_p: 19192.63|macdd: -9.27|p: None |c: False
l_t: 2022-10-19 07:08:12.056000|st: 2022-10-19 07:05:00|l_p: 19194.48|macdd: -9.14|p: None |c: False
l_t: 2022-10-19 07:08:14.119000|st: 2022-10-19 07:05:00|l_p: 19195.11|macdd: -9.10|p: None |c: False
l_t: 2022-10-19 07:08:16.165000|st: 2022-10-19 07:05:00|l_p: 19197.61|macdd: -8.92|p: None |c: False
l_t: 2022-10-19 07:08:18.246000|st: 2022-10-19 07:05:00|l_p: 19195.21|macdd: -9.09|p: None |c: False
l_t: 2022-10-19 07:08:20.311000|st: 2022-10-19 07:05:00|l_p: 19199.61|macdd: -8.78|p: None |c: False
l_t: 2022-10-19 07:08:22.325000|st: 2022-10-19 07:05:00|l_p: 19200.91|macdd: -8.69|p: No

l_t: 2022-10-19 07:10:52.002000|st: 2022-10-19 07:10:00|l_p: 19202.76|macdd: -7.50|p: None |c: False
l_t: 2022-10-19 07:10:54.006000|st: 2022-10-19 07:10:00|l_p: 19202.47|macdd: -7.52|p: None |c: False
l_t: 2022-10-19 07:10:56.013000|st: 2022-10-19 07:10:00|l_p: 19206.27|macdd: -7.25|p: None |c: False
l_t: 2022-10-19 07:10:58.027000|st: 2022-10-19 07:10:00|l_p: 19205.50|macdd: -7.31|p: None |c: False
l_t: 2022-10-19 07:11:00.027000|st: 2022-10-19 07:10:00|l_p: 19206.24|macdd: -7.26|p: None |c: False
l_t: 2022-10-19 07:11:02.141000|st: 2022-10-19 07:10:00|l_p: 19203.94|macdd: -7.42|p: None |c: False
l_t: 2022-10-19 07:11:04.214000|st: 2022-10-19 07:10:00|l_p: 19202.64|macdd: -7.51|p: None |c: False
l_t: 2022-10-19 07:11:06.422000|st: 2022-10-19 07:10:00|l_p: 19203.36|macdd: -7.46|p: None |c: False
l_t: 2022-10-19 07:11:08.697000|st: 2022-10-19 07:10:00|l_p: 19203.20|macdd: -7.47|p: None |c: False
l_t: 2022-10-19 07:11:10.788000|st: 2022-10-19 07:10:00|l_p: 19203.07|macdd: -7.48|p: None 

l_t: 2022-10-19 07:13:43.407000|st: 2022-10-19 07:10:00|l_p: 19216.65|macdd: -6.52|p: None |c: False
l_t: 2022-10-19 07:13:45.409000|st: 2022-10-19 07:10:00|l_p: 19216.52|macdd: -6.53|p: None |c: False
l_t: 2022-10-19 07:13:47.533000|st: 2022-10-19 07:10:00|l_p: 19214.05|macdd: -6.70|p: None |c: False
l_t: 2022-10-19 07:13:49.608000|st: 2022-10-19 07:10:00|l_p: 19215.36|macdd: -6.61|p: None |c: False
l_t: 2022-10-19 07:13:51.630000|st: 2022-10-19 07:10:00|l_p: 19214.00|macdd: -6.71|p: None |c: False
l_t: 2022-10-19 07:13:53.742000|st: 2022-10-19 07:10:00|l_p: 19214.00|macdd: -6.71|p: None |c: False
l_t: 2022-10-19 07:13:56.202000|st: 2022-10-19 07:10:00|l_p: 19212.75|macdd: -6.79|p: None |c: False
l_t: 2022-10-19 07:13:58.308000|st: 2022-10-19 07:10:00|l_p: 19214.02|macdd: -6.70|p: None |c: False
l_t: 2022-10-19 07:14:00.361000|st: 2022-10-19 07:10:00|l_p: 19213.39|macdd: -6.75|p: None |c: False
l_t: 2022-10-19 07:14:02.366000|st: 2022-10-19 07:10:00|l_p: 19214.32|macdd: -6.68|p: None 

l_t: 2022-10-19 07:16:34.644000|st: 2022-10-19 07:15:00|l_p: 19208.74|macdd: -5.43|p: None |c: False
l_t: 2022-10-19 07:16:36.718000|st: 2022-10-19 07:15:00|l_p: 19208.49|macdd: -5.45|p: None |c: False
l_t: 2022-10-19 07:16:38.746000|st: 2022-10-19 07:15:00|l_p: 19205.75|macdd: -5.65|p: None |c: False
l_t: 2022-10-19 07:16:40.872000|st: 2022-10-19 07:15:00|l_p: 19206.35|macdd: -5.60|p: None |c: False
l_t: 2022-10-19 07:16:42.918000|st: 2022-10-19 07:15:00|l_p: 19206.90|macdd: -5.57|p: None |c: False
l_t: 2022-10-19 07:16:45.032000|st: 2022-10-19 07:15:00|l_p: 19207.27|macdd: -5.54|p: None |c: False
l_t: 2022-10-19 07:16:47.225000|st: 2022-10-19 07:15:00|l_p: 19207.59|macdd: -5.52|p: None |c: False
l_t: 2022-10-19 07:16:49.338000|st: 2022-10-19 07:15:00|l_p: 19207.58|macdd: -5.52|p: None |c: False
l_t: 2022-10-19 07:16:51.476000|st: 2022-10-19 07:15:00|l_p: 19207.07|macdd: -5.55|p: None |c: False
l_t: 2022-10-19 07:16:53.719000|st: 2022-10-19 07:15:00|l_p: 19206.43|macdd: -5.60|p: None 

l_t: 2022-10-19 07:19:24.975000|st: 2022-10-19 07:15:00|l_p: 19226.67|macdd: -4.16|p: None |c: False
l_t: 2022-10-19 07:19:27.027000|st: 2022-10-19 07:15:00|l_p: 19227.01|macdd: -4.14|p: None |c: False
l_t: 2022-10-19 07:19:29.370000|st: 2022-10-19 07:15:00|l_p: 19226.80|macdd: -4.15|p: None |c: False
l_t: 2022-10-19 07:19:31.405000|st: 2022-10-19 07:15:00|l_p: 19227.83|macdd: -4.08|p: None |c: False
l_t: 2022-10-19 07:19:33.463000|st: 2022-10-19 07:15:00|l_p: 19227.93|macdd: -4.07|p: None |c: False
l_t: 2022-10-19 07:19:35.515000|st: 2022-10-19 07:15:00|l_p: 19228.12|macdd: -4.06|p: None |c: False
l_t: 2022-10-19 07:19:37.605000|st: 2022-10-19 07:15:00|l_p: 19228.03|macdd: -4.06|p: None |c: False
l_t: 2022-10-19 07:19:39.649000|st: 2022-10-19 07:15:00|l_p: 19228.41|macdd: -4.04|p: None |c: False
l_t: 2022-10-19 07:19:41.681000|st: 2022-10-19 07:15:00|l_p: 19229.31|macdd: -3.97|p: None |c: False
l_t: 2022-10-19 07:19:43.716000|st: 2022-10-19 07:15:00|l_p: 19229.31|macdd: -3.97|p: None 

...

l_t: 2022-10-20 02:38:46.570000|st: 2022-10-20 02:35:00|l_p: 19060.17|macdd: 13.84|p: None |c: False
l_t: 2022-10-20 02:38:48.589000|st: 2022-10-20 02:35:00|l_p: 19059.87|macdd: 13.82|p: None |c: False
l_t: 2022-10-20 02:38:50.600000|st: 2022-10-20 02:35:00|l_p: 19061.04|macdd: 13.90|p: None |c: False
l_t: 2022-10-20 02:38:53.113000|st: 2022-10-20 02:35:00|l_p: 19060.86|macdd: 13.89|p: None |c: False
l_t: 2022-10-20 02:38:55.236000|st: 2022-10-20 02:35:00|l_p: 19061.12|macdd: 13.91|p: None |c: False
l_t: 2022-10-20 02:38:57.242000|st: 2022-10-20 02:35:00|l_p: 19062.44|macdd: 14.00|p: None |c: False
l_t: 2022-10-20 02:38:59.253000|st: 2022-10-20 02:35:00|l_p: 19062.69|macdd: 14.02|p: None |c: False
l_t: 2022-10-20 02:39:01.368000|st: 2022-10-20 02:35:00|l_p: 19062.81|macdd: 14.03|p: None |c: False
l_t: 2022-10-20 02:39:03.375000|st: 2022-10-20 02:35:00|l_p: 19063.04|macdd: 14.05|p: None |c: False
l_t: 2022-10-20 02:39:05.425000|st: 2022-10-20 02:35:00|l_p: 19062.40|macdd: 14.00|p: None 

l_t: 2022-10-20 02:41:40.446000|st: 2022-10-20 02:40:00|l_p: 19048.78|macdd: 12.25|p: None |c: False
l_t: 2022-10-20 02:41:42.579000|st: 2022-10-20 02:40:00|l_p: 19049.59|macdd: 12.31|p: None |c: False
l_t: 2022-10-20 02:41:44.627000|st: 2022-10-20 02:40:00|l_p: 19049.19|macdd: 12.28|p: None |c: False
l_t: 2022-10-20 02:41:47.039000|st: 2022-10-20 02:40:00|l_p: 19049.12|macdd: 12.27|p: None |c: False
l_t: 2022-10-20 02:41:49.123000|st: 2022-10-20 02:40:00|l_p: 19048.71|macdd: 12.25|p: None |c: False
l_t: 2022-10-20 02:41:51.155000|st: 2022-10-20 02:40:00|l_p: 19047.23|macdd: 12.14|p: None |c: False
l_t: 2022-10-20 02:41:53.202000|st: 2022-10-20 02:40:00|l_p: 19047.35|macdd: 12.15|p: None |c: False
l_t: 2022-10-20 02:41:55.338000|st: 2022-10-20 02:40:00|l_p: 19047.36|macdd: 12.15|p: None |c: False
l_t: 2022-10-20 02:41:57.408000|st: 2022-10-20 02:40:00|l_p: 19048.70|macdd: 12.24|p: None |c: False
l_t: 2022-10-20 02:41:59.423000|st: 2022-10-20 02:40:00|l_p: 19047.97|macdd: 12.19|p: None 

l_t: 2022-10-20 02:44:31.826000|st: 2022-10-20 02:40:00|l_p: 19053.70|macdd: 12.60|p: None |c: False
l_t: 2022-10-20 02:44:33.910000|st: 2022-10-20 02:40:00|l_p: 19052.80|macdd: 12.54|p: None |c: False
l_t: 2022-10-20 02:44:36.050000|st: 2022-10-20 02:40:00|l_p: 19052.39|macdd: 12.51|p: None |c: False
l_t: 2022-10-20 02:44:38.064000|st: 2022-10-20 02:40:00|l_p: 19052.31|macdd: 12.50|p: None |c: False
l_t: 2022-10-20 02:44:40.102000|st: 2022-10-20 02:40:00|l_p: 19053.30|macdd: 12.57|p: None |c: False
l_t: 2022-10-20 02:44:42.135000|st: 2022-10-20 02:40:00|l_p: 19055.46|macdd: 12.72|p: None |c: False
l_t: 2022-10-20 02:44:44.279000|st: 2022-10-20 02:40:00|l_p: 19055.47|macdd: 12.73|p: None |c: False
l_t: 2022-10-20 02:44:46.497000|st: 2022-10-20 02:40:00|l_p: 19054.87|macdd: 12.68|p: None |c: False
l_t: 2022-10-20 02:44:48.894000|st: 2022-10-20 02:40:00|l_p: 19054.93|macdd: 12.69|p: None |c: False
l_t: 2022-10-20 02:44:50.969000|st: 2022-10-20 02:40:00|l_p: 19055.17|macdd: 12.70|p: None 

l_t: 2022-10-20 02:47:26.679000|st: 2022-10-20 02:45:00|l_p: 19036.76|macdd: 10.76|p: None |c: False
l_t: 2022-10-20 02:47:28.683000|st: 2022-10-20 02:45:00|l_p: 19042.92|macdd: 11.20|p: None |c: False
l_t: 2022-10-20 02:47:30.747000|st: 2022-10-20 02:45:00|l_p: 19042.48|macdd: 11.17|p: None |c: False
l_t: 2022-10-20 02:47:32.876000|st: 2022-10-20 02:45:00|l_p: 19042.48|macdd: 11.17|p: None |c: False
l_t: 2022-10-20 02:47:34.918000|st: 2022-10-20 02:45:00|l_p: 19045.14|macdd: 11.36|p: None |c: False
l_t: 2022-10-20 02:47:37.146000|st: 2022-10-20 02:45:00|l_p: 19043.46|macdd: 11.24|p: None |c: False
l_t: 2022-10-20 02:47:39.728000|st: 2022-10-20 02:45:00|l_p: 19043.92|macdd: 11.27|p: None |c: False
l_t: 2022-10-20 02:47:41.768000|st: 2022-10-20 02:45:00|l_p: 19044.21|macdd: 11.29|p: None |c: False
l_t: 2022-10-20 02:47:43.864000|st: 2022-10-20 02:45:00|l_p: 19044.11|macdd: 11.28|p: None |c: False
l_t: 2022-10-20 02:47:45.959000|st: 2022-10-20 02:45:00|l_p: 19043.90|macdd: 11.27|p: None 

l_t: 2022-10-20 02:50:21.503000|st: 2022-10-20 02:50:00|l_p: 19043.33|macdd: 10.04|p: None |c: False
l_t: 2022-10-20 02:50:23.650000|st: 2022-10-20 02:50:00|l_p: 19042.82|macdd: 10.00|p: None |c: False
l_t: 2022-10-20 02:50:25.705000|st: 2022-10-20 02:50:00|l_p: 19043.63|macdd: 10.06|p: None |c: False
l_t: 2022-10-20 02:50:27.809000|st: 2022-10-20 02:50:00|l_p: 19043.34|macdd: 10.04|p: None |c: False
l_t: 2022-10-20 02:50:30.031000|st: 2022-10-20 02:50:00|l_p: 19043.06|macdd: 10.02|p: None |c: False
l_t: 2022-10-20 02:50:32.126000|st: 2022-10-20 02:50:00|l_p: 19042.72|macdd: 10.00|p: None |c: False
l_t: 2022-10-20 02:50:34.181000|st: 2022-10-20 02:50:00|l_p: 19042.52|macdd: 9.98|p: None |c: False
l_t: 2022-10-20 02:50:36.188000|st: 2022-10-20 02:50:00|l_p: 19043.10|macdd: 10.02|p: None |c: False
l_t: 2022-10-20 02:50:38.395000|st: 2022-10-20 02:50:00|l_p: 19042.72|macdd: 10.00|p: None |c: False
l_t: 2022-10-20 02:50:41.049000|st: 2022-10-20 02:50:00|l_p: 19043.65|macdd: 10.06|p: None |

In [46]:
trader_test.stop_ses(save_to_file=False)

trading sesion duration = 110.0 minutes up to 2880
STOP
l_t: 2022-10-20 08:57:54.544000|st: 2022-10-20 08:55:00|l_p: 19118.50|macdd: -4.00|p: None |c: False


In [32]:
#trader_test.save_to_file()

In [44]:
trader_test.trade_values

[-961.3185,
 958.517,
 -959.755,
 959.1295,
 -958.77960536,
 956.19101039,
 -958.6965,
 960.27,
 -960.2085,
 958.849,
 -960.4045,
 959.436,
 -960.2775,
 959.8185,
 -958.062,
 954.3965,
 -951.841,
 956.4595,
 -957.259,
 956.9505]

l_t: 2022-10-20 08:57:37.936000|st: 2022-10-20 08:55:00|l_p: 19121.93|macdd: -3.76|p: None |c: False
l_t: 2022-10-20 08:57:39.956000|st: 2022-10-20 08:55:00|l_p: 19121.51|macdd: -3.79|p: None |c: False
l_t: 2022-10-20 08:57:41.970000|st: 2022-10-20 08:55:00|l_p: 19121.42|macdd: -3.79|p: None |c: False
l_t: 2022-10-20 08:57:44.034000|st: 2022-10-20 08:55:00|l_p: 19119.65|macdd: -3.92|p: None |c: False


In [45]:
trader_test.cum_profits

-6.585

l_t: 2022-10-20 08:57:46.289000|st: 2022-10-20 08:55:00|l_p: 19119.87|macdd: -3.90|p: None |c: False
l_t: 2022-10-20 08:57:48.308000|st: 2022-10-20 08:55:00|l_p: 19119.37|macdd: -3.94|p: None |c: False
l_t: 2022-10-20 08:57:50.446000|st: 2022-10-20 08:55:00|l_p: 19118.69|macdd: -3.99|p: None |c: False
l_t: 2022-10-20 08:57:52.538000|st: 2022-10-20 08:55:00|l_p: 19118.61|macdd: -3.99|p: None |c: False


In [81]:
#trader_test.plot_results(width_bars=0.0003)

In [80]:
#trader_test.data

In [19]:
#round(float(trader_peak_test.client.get_asset_balance(asset='USDT')['free']),3)

In [79]:
#trader_test.initial_balance_USDT

In [78]:
#trader_test.final_balance_USDT

In [77]:
#trader_test.client.get_account()

In [24]:
#trader_test.open_orders()

In [16]:
#trader_test.trade_start_time_utc

In [17]:
#trader_test.trade_values

In [18]:
#trader_test.initial_balance_USDT

In [19]:
#trader_test.final_balance_USDT